# Chapter 16: Asynchronous Programming with `async`/`await`

Modern applications often need to perform operations that take time: reading files, querying databases, downloading web resources, or waiting for user input. If you execute these operations synchronously (blocking the current thread), your application becomes unresponsive. Users experience freezing, and server applications waste threads that could be serving other requests.

**Asynchronous programming** allows you to start a long‑running operation and then continue executing other work without blocking the calling thread. When the operation completes, your code resumes. C#’s `async` and `await` keywords, introduced in C# 5, make asynchronous code almost as easy to write and read as synchronous code.

In this chapter, you'll learn:

- The **Task‑based Asynchronous Pattern (TAP)** and the role of `Task` and `Task<T>`.
- How to use the `async` and `await` keywords to write non‑blocking code.
- The **compiler transformation** that turns async methods into state machines.
- Best practices for async programming (avoid `async void`, use `ConfigureAwait`, etc.).
- **Cancellation** with `CancellationToken`.
- Combining multiple asynchronous operations with `Task.WhenAll` and `Task.WhenAny`.
- **Error handling** in async methods.
- A practical example: downloading web pages concurrently.

By the end, you'll be able to build responsive applications that efficiently handle I/O‑bound and CPU‑bound work.

---

## 16.1 The Problem with Blocking Calls

Consider a simple program that downloads a web page:

```csharp
public string DownloadWebPage(string url)
{
    using (var client = new WebClient())
    {
        return client.DownloadString(url); // Blocks the current thread
    }
}
```

When `DownloadString` is called, the thread waits for the network operation to complete. During that time, the thread cannot do anything else. In a desktop application, this freezes the UI. In a server application, it wastes a thread that could handle other requests.

Asynchronous programming solves this by not blocking the thread. Instead, the operation runs in the background, and the thread returns to the thread pool to do other work. When the operation completes, the rest of the code continues.

---

## 16.2 The Task‑based Asynchronous Pattern (TAP)

The foundation of async programming in .NET is the `Task` class (and its generic counterpart `Task<TResult>`). A `Task` represents an **ongoing operation** that may complete in the future. It can be awaited, combined, and used to check status, retrieve results, or handle exceptions.

### `Task` and `Task<TResult>`

- `Task` – represents an operation that does not return a value.
- `Task<TResult>` – represents an operation that returns a value of type `TResult`.

You can create a task using `Task.Run` for CPU‑bound work, or use library methods that already return tasks (e.g., `HttpClient.GetStringAsync`).

```csharp
// CPU‑bound work (runs on a thread pool thread)
Task<int> task = Task.Run(() => ComputeSomething());

// I/O‑bound work (uses true asynchronous I/O, no thread blocked)
HttpClient client = new HttpClient();
Task<string> downloadTask = client.GetStringAsync("https://example.com");
```

---

## 16.3 The `async` and `await` Keywords

The `async` keyword marks a method as asynchronous. It enables the use of `await` inside the method. The `await` operator is applied to a task, telling the compiler to suspend the method until the task completes, but without blocking the current thread.

### Basic Example

```csharp
public async Task<string> DownloadWebPageAsync(string url)
{
    using (HttpClient client = new HttpClient())
    {
        // This line does NOT block; it returns a Task<string> that will complete later
        string content = await client.GetStringAsync(url);
        // After the await, execution resumes here when the download is done
        return content;
    }
}
```

**What happens when `await` is reached?**

1. The method checks if the task has already completed. If so, it continues synchronously.
2. If not, the method suspends and returns an incomplete task to the caller. The current thread is free to do other work.
3. When the awaited task completes, the remainder of the method is scheduled to run (typically on the same context, e.g., UI thread).

### Calling an Async Method

```csharp
public async Task CallerMethod()
{
    string result = await DownloadWebPageAsync("https://example.com");
    Console.WriteLine(result);
}
```

Notice that `CallerMethod` is also `async` and returns a `Task`. In practice, you'll often have a chain of async methods. At the top level (e.g., a button click handler), you can use `async void` (more on that later) or, in a console app's `Main`, you can use `Task.Wait` (but avoid it in production code).

### Return Types for Async Methods

- **`Task`** – for an async method that performs work but returns no value.
- **`Task<T>`** – for an async method that returns a value of type `T`.
- **`void`** – only for event handlers (e.g., button click handlers). Avoid `async void` elsewhere because exceptions cannot be caught by the caller.

```csharp
public async Task DoWorkAsync() // returns Task
{
    await Task.Delay(1000);
}

public async Task<int> GetResultAsync() // returns Task<int>
{
    await Task.Delay(1000);
    return 42;
}

public async void Button_Click(object sender, EventArgs e) // event handler
{
    await DoWorkAsync();
}
```

---

## 16.4 How Async/Await Works: The Compiler Transformation

The compiler transforms an async method into a **state machine**. The code is broken into chunks, each representing a section of the method between `await` expressions. When an `await` is reached, the method returns an incomplete task. When the awaited task completes, the state machine resumes execution from where it left off.

This transformation is complex, but you don't need to understand all the details. Just know that it's efficient and doesn't create extra threads for I/O operations. For I/O, the underlying API (like `HttpClient.GetStringAsync`) uses true asynchronous I/O – no thread is blocked during the operation.

---

## 16.5 Best Practices for Async Programming

### 1. Avoid `async void`

`async void` methods have a different error‑handling semantics: exceptions thrown inside them are raised on the synchronization context (often crashing the process). They are only acceptable for event handlers. For all other methods, return `Task` or `Task<T>`.

### 2. Use `ConfigureAwait(false)` in Library Code

When you `await` a task, by default the continuation is marshalled back to the original context (e.g., the UI thread). In library code that doesn't need to return to a specific context, you can use `ConfigureAwait(false)` to avoid this overhead and prevent deadlocks.

```csharp
await SomeOperationAsync().ConfigureAwait(false);
```

### 3. Name Async Methods with the "Async" Suffix

By convention, async methods that return `Task` or `Task<T>` should end with "Async". This makes it clear that the method is awaitable.

### 4. Don't Block on Async Code

Avoid using `.Result` or `.Wait()` on a task, as this can cause deadlocks (especially in UI apps). Instead, use `await` all the way up.

```csharp
// Bad
string result = DownloadWebPageAsync("...").Result;

// Good
string result = await DownloadWebPageAsync("...");
```

If you must call an async method from a synchronous method (e.g., in a console app's `Main` before C# 7.1), you can use `GetAwaiter().GetResult()` but be aware of deadlock risks. In modern C# (7.1+), `Main` can be `async Task`.

### 5. Use `Task.WhenAll` and `Task.WhenAny` for Concurrency

Don't `await` tasks sequentially if they can run concurrently. Use `Task.WhenAll` to wait for multiple tasks to complete.

### 6. Handle Exceptions Properly

Exceptions thrown in async methods are captured by the returned task. When you `await` the task, the exception is rethrown. You can catch it with `try`/`catch` as usual.

### 7. Pass `CancellationToken` When Possible

Many async methods accept a `CancellationToken` to support cancellation. Provide one to make your code more responsive and avoid unnecessary work.

---

## 16.6 Cancellation Tokens

**Cancellation tokens** allow you to signal that an operation should be cancelled. They follow a cooperative pattern: the operation checks the token periodically and stops if cancellation is requested.

### Using `CancellationTokenSource` and `CancellationToken`

```csharp
CancellationTokenSource cts = new CancellationTokenSource();
CancellationToken token = cts.Token;

// Start an operation that supports cancellation
Task task = DoLongWorkAsync(token);

// Later, if you want to cancel:
cts.Cancel();
```

In the async method, you can check the token:

```csharp
public async Task DoLongWorkAsync(CancellationToken token)
{
    for (int i = 0; i < 100; i++)
    {
        token.ThrowIfCancellationRequested(); // throws OperationCanceledException
        await Task.Delay(100, token); // many built‑in methods accept a token
    }
}
```

When cancellation is requested, `ThrowIfCancellationRequested` throws an `OperationCanceledException`. This exception should be caught by the caller to handle cancellation gracefully.

### Cancelling a `Task` with `CancellationToken`

```csharp
try
{
    await DoLongWorkAsync(cts.Token);
}
catch (OperationCanceledException)
{
    Console.WriteLine("Operation was cancelled.");
}
```

Cancellation is cooperative – the operation must be designed to respond to the token.

---

## 16.7 Combining Tasks: `Task.WhenAll` and `Task.WhenAny`

### `Task.WhenAll`

Waits for **all** provided tasks to complete. It returns a single task that completes when all tasks are done.

```csharp
Task<string> task1 = DownloadAsync("url1");
Task<string> task2 = DownloadAsync("url2");
Task<string> task3 = DownloadAsync("url3");

string[] results = await Task.WhenAll(task1, task2, task3);
// results contains the results in the same order
```

If any task faults, the returned task faults, and you can examine the aggregated exceptions.

### `Task.WhenAny`

Waits for **any** of the provided tasks to complete. It returns the first completed task.

```csharp
Task<string> task1 = DownloadAsync("url1");
Task<string> task2 = DownloadAsync("url2");

Task<string> firstCompleted = await Task.WhenAny(task1, task2);
string result = await firstCompleted; // you still need to await to get the result
Console.WriteLine(result);
```

`WhenAny` is useful for implementing timeouts or race conditions.

---

## 16.8 Error Handling in Async Methods

Exceptions in async methods are stored in the returned task. When you `await` the task, the exception is rethrown at that point. You can catch it with `try`/`catch` around the `await`.

```csharp
try
{
    await DoSomethingAsync();
}
catch (SpecificException ex)
{
    // handle
}
```

If you have multiple tasks with `Task.WhenAll`, you can catch `AggregateException` or catch the specific exceptions (they are unwrapped).

```csharp
try
{
    await Task.WhenAll(task1, task2);
}
catch (Exception ex) // catches the first exception (unwrapped)
{
    Console.WriteLine(ex.Message);
}
```

To get all exceptions, you can access the `Exception` property of the faulted task.

---

## 16.9 Putting It All Together: A Practical Example

Let's build a program that downloads multiple web pages concurrently, with cancellation support and error handling.

```csharp
using System;
using System.Collections.Generic;
using System.Net.Http;
using System.Threading;
using System.Threading.Tasks;

namespace AsyncDemo
{
    class Program
    {
        static async Task Main(string[] args)
        {
            var urls = new List<string>
            {
                "https://example.com",
                "https://httpbin.org/delay/2",
                "https://httpbin.org/status/404",
                "https://nonexistent.url.abc" // will cause an exception
            };

            using var cts = new CancellationTokenSource();
            // Simulate user cancelling after 3 seconds
            cts.CancelAfter(TimeSpan.FromSeconds(3));

            try
            {
                var results = await DownloadAllPagesAsync(urls, cts.Token);
                foreach (var (url, content) in results)
                {
                    Console.WriteLine($"--- {url} ---");
                    Console.WriteLine(content.Substring(0, Math.Min(100, content.Length)));
                }
            }
            catch (OperationCanceledException)
            {
                Console.WriteLine("Download was cancelled.");
            }
            catch (Exception ex)
            {
                Console.WriteLine($"An error occurred: {ex.Message}");
            }
        }

        static async Task<List<(string Url, string Content)>> DownloadAllPagesAsync(
            List<string> urls, CancellationToken cancellationToken)
        {
            using var httpClient = new HttpClient();
            var tasks = new List<Task<(string Url, string Content)>>();

            foreach (var url in urls)
            {
                tasks.Add(DownloadPageAsync(httpClient, url, cancellationToken));
            }

            var results = await Task.WhenAll(tasks);
            return new List<(string, string)>(results);
        }

        static async Task<(string Url, string Content)> DownloadPageAsync(
            HttpClient client, string url, CancellationToken token)
        {
            try
            {
                string content = await client.GetStringAsync(url, token);
                return (url, content);
            }
            catch (Exception ex)
            {
                // Return a tuple with error message instead of throwing
                return (url, $"Error: {ex.Message}");
            }
        }
    }
}
```

**Explanation:**

- `Main` is `async Task` (C# 7.1+), allowing us to `await` directly.
- We create a `CancellationTokenSource` that cancels after 3 seconds to simulate a timeout.
- `DownloadAllPagesAsync` creates a list of tasks, one per URL, using `HttpClient.GetStringAsync`. Each task returns a tuple with the URL and the content (or error message).
- We use `Task.WhenAll` to wait for all downloads concurrently.
- Individual download exceptions are caught inside `DownloadPageAsync` and returned as error messages, so one failing download doesn't stop the others.
- The result is a list of tuples that we can process.

This example demonstrates concurrency, cancellation, error handling, and returning structured results from async methods.

---

## 16.10 Common Pitfalls and Best Practices – Recap

- **Deadlocks** – avoid blocking on async code with `.Result` or `.Wait()`, especially in UI or ASP.NET contexts. Use `await` all the way.
- **Async over sync** – don't wrap synchronous code in `Task.Run` just to make it async; that uses unnecessary threads. Use true async APIs.
- **Sync over async** – avoid wrapping async code in a synchronous method; it defeats the purpose.
- **Resource cleanup** – use `await using` for `IAsyncDisposable` or `using` for synchronous disposables inside async methods.
- **Thread pool starvation** – when you have many blocked tasks, the thread pool may not inject new threads quickly enough. Use async I/O to avoid blocking.
- **CancellationToken** – always propagate it to inner async calls if possible.

---

## 16.11 Chapter Summary

In this chapter, you've learned how to write responsive, non‑blocking code using `async` and `await`:

- **Tasks** represent ongoing operations.
- **`async`** marks a method as asynchronous; **`await`** suspends the method until a task completes without blocking.
- The compiler transforms async methods into efficient state machines.
- **Best practices** include avoiding `async void`, using `ConfigureAwait(false)` in libraries, naming methods with "Async", and never blocking on async code.
- **Cancellation tokens** enable cooperative cancellation.
- **`Task.WhenAll`** and **`Task.WhenAny`** let you compose tasks for concurrency.
- **Error handling** in async methods works with `try`/`catch` around `await`.

Asynchronous programming is essential for modern applications. Mastering it will make your code more responsive, scalable, and efficient.

In the next chapter, **Pattern Matching in Depth**, we'll explore C#'s powerful pattern matching capabilities. You'll learn how to write concise, expressive code for type testing, property matching, and more, using `is` expressions, `switch` expressions, and pattern combinators.

**Exercises:**

1. Write an async method that downloads multiple images from a list of URLs concurrently. Use `Task.WhenAll`. Measure the time with and without concurrency.
2. Implement a timeout for an async operation using `CancellationToken` and `CancelAfter`. Simulate a long‑running operation with `Task.Delay`.
3. Create a simple UI application (WPF or WinForms) with a button that starts an async operation and updates a progress bar. Handle cancellation.
4. Write a method that retries an async operation up to 3 times with exponential backoff.
5. Use `Task.WhenAny` to implement a "race" between two data sources: whichever responds first wins.

Now, get ready to unlock the expressive power of pattern matching in Chapter 17!